In [1]:
# Example of simple model push to Acumos/CMLP

# ===============LICENSE_START=======================================================
# Apache-2.0
# ===================================================================================
# Copyright (C) 2019 AT&T Intellectual Property  All rights reserved.
# ===================================================================================
# This software file is distributed by AT&T
# under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# This file is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ===============LICENSE_END=========================================================


In [2]:
import numpy as np
import pandas as pd
import os,sys,shutil  # file checks
import dill as pickle   # serialize functions and data as compressed binary 
import gzip  # compression 
import yaml   # configuration file
import time  # time tracking

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.session import AcumosSession, Requirements

# load our configutaion
config_path = 'config.yaml'
if not os.path.isfile(config_path):
    print("Sorry, can't find the configuration file {}, aborting.".format(config_path))
    sys.exit(-1)
config = yaml.safe_load(open(config_path))

In [63]:
## PART 1 - load models and dump to disk 
# https://pypi.org/project/acumos/#using-dataframes-with-scikit-learn

# load our models
with gzip.open(config["path"]["model_preproc"], 'rb') as f:
    etl = pickle.load(f)
with gzip.open(config["path"]["model_classifier"], 'rb') as f:
    models = pickle.load(f)

# here, an appropriate NamedTuple type is inferred from a pandas DataFrame
print(list(etl["fn_input"].columns))
print(list(etl["fn_input"].dtypes))

ReviewDataFrame = create_namedtuple('ReviewDataFrame', 
    [('helpful', List[int]), ("reviewText", str), ("summary", str),
     ("unixReviewTime", int), ("categories", List[str]), ("description", str)])
# ClassifyDataFrame = create_dataframe('ReviewDataFrame', etl["fn_input"])
print(ReviewDataFrame)
ReviewPredictions = create_namedtuple(
    'ReviewPredictions', [("prob{}".format(n), float) for n in models["best"].classes_])

def classify_review(df: ReviewDataFrame) -> ReviewPredictions:
    '''Returns an array of review classifications'''
    X = pd.DataFrame([df], columns=ReviewDataFrame._fields)  # we need this because of custom types
    # X = np.column_stack(df)
    X_etl = etl["fn_preproc"](etl, X)  # do classification, don't modify our model
    X_res = models["best"].predict_proba(X_etl)[0]
    # print(models["best"].classes_)
    # print(X_res)
    return ReviewPredictions(*X_res)

model = Model(classify=classify_review)



['helpful', 'reviewText', 'summary', 'unixReviewTime', 'categories', 'description']
[dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('O'), dtype('O')]
<class 'acumos.modeling.ReviewDataFrame'>


In [64]:
# create model so that we can run it locally
session = AcumosSession( config["publish"]["url_model"],  config["publish"]["url_auth"])
path_dump = os.path.join('data', config["publish"]["name_model2"])
if os.path.exists(path_dump):
    shutil.rmtree(path_dump)
session.dump(model, config["publish"]["name_model2"], 'data')  # creates ~/<name_publish>

In [56]:
## PART 2 - publish models
os.environ['ACUMOS_TOKEN'] = config["publish"]["api_token"] # visible in this process + all children

# just grab the local scripts for requirements, look at the URL below for more examples
#    of complicated requirements  - https://pypi.org/project/acumos/#declaring-requirements
reqs = Requirements()

# using the AcumosSession created earlier:
time_start = time.time()
session.push(model, config["publish"]["name_model2"], reqs)
print("Model push time... {:0.3f} sec".format(time.time()-time_start))


[WARNING] acumos.session : Provided `push_api` API http://acumosr-test.research.att.com:8090/onboarding-app/v2/models does not begin with 'https'. Your password and token are visible in plaintext!
[WARNING] acumos.session : Provided `auth_api` API http://acumosr-test.research.att.com:8090/onboarding-app/v2/auth does not begin with 'https'. Your password and token are visible in plaintext!


AcumosError: Multiple definitions found for type ReviewPredictions: [<class 'acumos.modeling.ReviewPredictions'>, <class 'acumos.modeling.ReviewPredictions'>]